<a href="https://colab.research.google.com/github/IverMartinsen/MastersThesis/blob/main/Notebooks/random_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook show a random search for hyperparameters.

In [ ]:
!pip install -q -U keras-tuner

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU
from tensorflow.keras.layers import MaxPool2D, Dropout, Dense, Flatten
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import tensorflow as tf
import kerastuner as kt
import numpy as np

def build_model(hp):
    
    L2 = tf.keras.regularizers.L2(
        l2=hp.Float("l2_norm", 1e-5, 1e-1, sampling="log"))
    
    inputs = Input(shape=(128, 128, 1))

    x = Rescaling(1./255)(inputs)

    filter_set = [8, 16, 24, 32, 48, 64, 96, 128]

    for i in range(hp.Int("num_conv_blocks", 3, 5)):
        
        filters = hp.Choice("conv_filters_" + str(i+1), filter_set[i:i+4])

        for j in range(hp.Int("block_size_" + str(i+1), 1, 2)):
            
            x = Conv2D(filters, 3, padding='same', kernel_regularizer=L2)(x)
            x = BatchNormalization()(x)
            x = ReLU()(x)
        
        x = MaxPool2D()(x)
        
    x = Flatten()(x)
    
    for i in range(hp.Int("dense_blocks", 1, 2)):
        
        x = Dense(
            hp.Int("dense_block_" + str(i+1), 8, 128, 8), 
            activation='relu', 
            kernel_regularizer=L2
            )(x)
        
        if i == 0:
            x = Dropout(hp.Float("dropout", 0, 0.5, 0.1))(x)
  
    outputs = Dense(1, activation='sigmoid', kernel_regularizer=L2)(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float("learning_rate", 1e-4, 1e-1, sampling="log")),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()],
    )
    
    return model

# import training and validation data
path = '/content/drive/MyDrive/Masteroppgave/Data/Torskeotolitter/standard_convex'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    label_mode='binary',
    validation_split=0.4,
    subset="training",
    seed=123,
    image_size=(128, 128),
    color_mode='grayscale',
    batch_size=32)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    label_mode='binary',
    validation_split=0.4,
    subset="validation",
    seed=123,
    image_size=(128, 128),
    color_mode='grayscale',
    batch_size=32)

# define parameter tuner instance
tuner = kt.RandomSearch(
    build_model,
    objective="val_binary_accuracy",
    max_trials=100, 
    project_name='random_search_4')

# search for parameters
tuner.search(
    train_ds, 
    epochs=1000, 
    validation_data=valid_ds,
    callbacks = [tf.keras.callbacks.EarlyStopping(
        patience=20, restore_best_weights=True)])

# save best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
model.save('/content/drive/MyDrive/Masteroppgave/best_convex_model')

Trial 100 Complete [00h 01m 54s]
val_binary_accuracy: 0.561475396156311

Best val_binary_accuracy So Far: 0.8319672346115112
Total elapsed time: 03h 13m 35s
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Masteroppgave/best_convex_model/assets
